In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
import utils
from bs4 import BeautifulSoup

In [2]:
products = pd.read_csv('data/DreamMarket_master_product.csv')

C:\Users\morga\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# utils.plot_null_val_heatmap(products, 'Null Values of Product Dataset')

## Functions - Add to utils.py

## Review Raw Data

In [4]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130936 entries, 0 to 130935
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   seqNo             39473 non-null   float64
 1   product_name      130936 non-null  object 
 2   category          130936 non-null  object 
 3   description       130936 non-null  object 
 4   shipping_options  130936 non-null  object 
 5   keywords          130936 non-null  object 
 6   seller_name       130936 non-null  object 
 7   price             130936 non-null  object 
 8   payment_method    130936 non-null  object 
 9   sold_since        130936 non-null  object 
 10  ends_in           130936 non-null  object 
 11  quantity_sold     130936 non-null  object 
 12  quantity_left     130936 non-null  object 
 13  refundPolicy      130936 non-null  object 
 14  market_name       130936 non-null  object 
 15  ship_from         130932 non-null  object 
 16  ship_to           13

In [5]:
products.head()

,seqNo,product_name,category,description,shipping_options,keywords,seller_name,price,payment_method,sold_since,ends_in,quantity_sold,quantity_left,refundPolicy,market_name,ship_from,ship_to,idproduct
0,1.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,-,-,-,-,-,-,Dream,-,-,NaN
1,2.0,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,-,GWLOVE,$125,Escrow Yes,-,-,-,-,-,Dream,CANADA,Worldwide Worldwide,NaN
2,3.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,-,-,-,-,-,-,Dream,-,-,NaN
3,4.0,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,-,Hectorstuff,$1,Escrow Yes,-,-,-,-,-,Dream,dutch,Worldwide Worldwide,NaN
4,5.0,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,-,darkmarket,$1,Escrow Yes,-,-,-,-,-,Dream,Worldwide,Worldwide Worldwide,NaN


In [6]:
cols_to_review = ['sold_since', 'ends_in', 'quantity_sold', 'quantity_left', 'refundPolicy', 'keywords']
for x in cols_to_review:
    print('{}: {}'.format(x, products[x].unique()))

sold_since: ['-']
ends_in: ['-']
quantity_sold: ['-']
quantity_left: ['-']
refundPolicy: ['-']
keywords: ['-']


## Text Cleaning

In [7]:
products_v2 = products.drop(cols_to_review, axis=1)

In [8]:
products_v2['is_2016'] = products_v2['idproduct'].isnull()
products_v2['year_offered'] = products_v2['is_2016'].apply(lambda x: 2016 if True else 2017)
products_v2.drop('is_2016', axis=1, inplace=True)
products_v2['adj_price'] = products_v2['price'].apply(lambda x: utils.convert_price_to_float(x))

In [9]:
products_v2['category_revised'] = products_v2['category'].str.replace(r'\n', '')
products_v2['category_revised'] = products_v2['category_revised'].str.replace('\t', '')
# products_v2['category_revised'] = products_v2['category_revised'].apply(lambda x: utils.replace_dash_with_nan(x))
products_v2['category_str'] = products_v2['category_revised'].str.findall(r'[a-zA-Z ]')

products_v2['category_str'] = products_v2['category_str'].apply(lambda x: ''.join(x))
products_v2['category_str'] = products_v2['category_str'].apply(lambda x: utils.replace_empty_cell_with_nan(x))
products_v2['category_num'] = products_v2['category_revised'].str.findall(r'[0-9 ]')
products_v2['category_num'] = products_v2['category_num'].apply(lambda x: ''.join(x).replace(' ', ''))
products_v2['category_num'] = products_v2['category_num'].apply(lambda x: utils.replace_empty_cell_with_nan(x))

In [10]:
products_v2['category_num'] = products_v2['category_num'].apply(lambda x: str(x).split('$'))

In [11]:
products_v2.head()

,seqNo,product_name,category,description,shipping_options,seller_name,price,payment_method,market_name,ship_from,ship_to,idproduct,year_offered,adj_price,category_revised,category_str,category_num
0,1.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,Dream,-,-,NaN,2016,NaN,-,NaN,[nan]
1,2.0,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,GWLOVE,$125,Escrow Yes,Dream,CANADA,Worldwide Worldwide,NaN,2016,125.0,Drugs 41793,Drugs,[41793]
2,3.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,-,NaN,2016,NaN,-,NaN,[nan]
3,4.0,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,Hectorstuff,$1,Escrow Yes,Dream,dutch,Worldwide Worldwide,NaN,2016,1.0,Drugs 41804,Drugs,[41804]
4,5.0,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,darkmarket,$1,Escrow Yes,Dream,Worldwide,Worldwide Worldwide,NaN,2016,1.0,Digital Goods 29334,Digital Goods,[29334]


---
## Extract Info From Product Name

**Regex Examples:**
```
df['method_1'] = df['text'].str.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}')
df['method_2'] = df['text'].str.findall(r'(\d{1,2}) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (\d{2,4})')
df['method_3'] = df['text'].str.findall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]* (\d{1,2})[,]* (\d{2,4})')
df['method_4'] = df['text'].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]* \d{4}')
df['method_5'] = df['text'].str.findall(r'\d{1,2}[/-]\d{4}')
df['method_6'] = df['text'].str.findall(r'\d{4}')
```    

---
## Extract Info From Description


In [12]:
products_v2['prod_name_split'] = products_v2['product_name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9$%_]',' ', x))
products_v2['prod_name_split'] = products_v2['prod_name_split'].apply(lambda x: re.split('\s+', x))
products_v2['prod_name_split_test'] = products_v2['prod_name_split'].apply(lambda x: utils.replace_split_text(x, r'[^a-zA-Z0-9$%_]',' '))



In [23]:
products_v2.iloc[:15, :]

,seqNo,product_name,category,description,shipping_options,seller_name,price,payment_method,market_name,ship_from,...,category_revised,category_str,category_num,prod_name_split,prod_name_split_test,shipping_cleaned,shipping_costs,shipping_non_financial,desc_weight_grams,shipping_options_revised
0,1.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,Dream,-,...,-,NaN,[nan],"[503, Forwarding, failure, Privoxy, localhost, ]","[, 503, Forwarding, failure, Privoxy, localhos...",-,[],"[-, ]",[],-
1,2.0,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,GWLOVE,$125,Escrow Yes,Dream,CANADA,...,Drugs 41793,Drugs,[41793],"[OG, KUSH, 1, 2, OZ, $125, AAAA, ]","[, OG, KUSH, 1, 2, OZ, $125, AAAA, ]",$15.99 CANADA WIDE $20.99 UN...,"[$15.99, $20.99, $40]","[ , , , , , , , , , , , , , , , C, A, ...",[],$15.99 CANADA WIDE $20.99 UN...
2,3.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,...,-,NaN,[nan],"[Dream, Market, Login, Featured, anonymous, ma...","[, Dream, Market, Login, Featured, anonymous, ...",-,[],"[-, ]",[],-
3,4.0,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,Hectorstuff,$1,Escrow Yes,Dream,dutch,...,Drugs 41804,Drugs,[41804],"[Every, drug, available, on, request]","[, Every, drug, available, on, request, ]",-,[],"[-, ]",[],-
4,5.0,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,darkmarket,$1,Escrow Yes,Dream,Worldwide,...,Digital Goods 29334,Digital Goods,[29334],"[Debug, Learn, how, crack, windows]","[, Debug, Learn, how, crack, windows, ]",-,[],"[-, ]",[],-
5,6.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,Dream,-,...,-,NaN,[nan],"[503, Forwarding, failure, Privoxy, localhost, ]","[, 503, Forwarding, failure, Privoxy, localhos...",-,[],"[-, ]",[],-
6,7.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,Dream,-,...,-,NaN,[nan],"[503, Forwarding, failure, Privoxy, localhost, ]","[, 503, Forwarding, failure, Privoxy, localhos...",-,[],"[-, ]",[],-
7,8.0,Ketamine HIGH QUALITY(Pharma S-Isomer) 5g,Drugs \n\t\t40803,PRODUCT:\nThis Pharma S-isomer ketamine. Lab ...,\n\n\n$3\t\t\t\t\t\nexpress\n\n\n\n\n\n,CrownBay,$500,Escrow Yes,Dream,AU,...,Drugs 40803,Drugs,[40803],"[Ketamine, HIGH, QUALITY, Pharma, S, Isomer, 5g]","[, Ketamine, HIGH, QUALITY, Pharma, S, Isomer,...",$3 express,[$3],"[ , , , , , , , , , , , e, x, p, r, e,...",[],$3 express
8,9.0,WIFI HACK,Services \n\t\t2220,Crack Any WiFi Password for\nHacking WEP/WPA/W...,\n\n\n$0\t\t\t\t\t\nDigital\n\n\n\n\n\n,BESTSHOP,$3.65,Escrow Yes,Dream,WW,...,Services 2220,Services,[2220],"[WIFI, HACK, ]","[, WIFI, HACK, ]",$0 Digital,[$0],"[ , , , , , , , , , , , D, i, g, i, t,...",[],$0 Digital
9,10.0,Hacked NETFLIX Account,Digital Goods \n\t\t29294,Hacked NETFLIX Account\n\nDon't change the pas...,-,Hackyboy,$8,Escrow Yes,Dream,pm,...,Digital Goods 29294,Digital Goods,[29294],"[Hacked, NETFLIX, Account]","[, Hacked, NETFLIX, Account, ]",-,[],"[-, ]",[],-


--- 
## Extract Info From Shipping

In [14]:
# Is it better to delay the removal of "\n" and "\t"?
products_v2['shipping_cleaned'] = products_v2['shipping_options'].str.replace(r'\n', ' ')
products_v2['shipping_cleaned'] = products_v2['shipping_cleaned'].str.replace('\t', ' ')
products_v2['shipping_costs'] = products_v2['shipping_cleaned'].str.findall(r'\$\d+(?:\.\d+)?')
products_v2['shipping_non_financial'] = products_v2['shipping_cleaned'].str.findall(r'[^\$\d+(?:\.\d+)]?')

In [15]:
products_v2.iloc[15:25, :]

,seqNo,product_name,category,description,shipping_options,seller_name,price,payment_method,market_name,ship_from,...,year_offered,adj_price,category_revised,category_str,category_num,prod_name_split,prod_name_split_test,shipping_cleaned,shipping_costs,shipping_non_financial
15,16.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,...,2016,NaN,-,NaN,[nan],"[Dream, Market, Login, Featured, anonymous, ma...","[, Dream, Market, Login, Featured, anonymous, ...",-,[],"[-, ]"
16,17.0,Fluor-Modafinil 100g,Drugs \n\t\t41830,Lab tested product purity around 98%.\nDo not ...,\n\n\n$39\t\t\t\t\t\nEMS\n\n\n\n\n\n$59\t\t\t\...,ROCKETCHEM,$490,Escrow Yes,Dream,China,...,2016,490.00,Drugs 41830,Drugs,[41830],"[Fluor, Modafinil, 100g]","[, Fluor, Modafinil, 100g, ]",$39 EMS $59 Fedex/TNT,"[$39, $59]","[ , , , , , , , , , , , , E, M, S, , ..."
17,18.0,Listing not found,-,-,-,-,-,-,Dream,-,...,2016,NaN,-,NaN,[nan],"[Listing, not, found]","[, Listing, not, found, ]",-,[],"[-, ]"
18,19.0,7g ORGANIC Maroc Hashish (sativa),Drugs \n\t\t42072,ONLY 9.99 per GRAM !!!\n\nOrganic Moroccan has...,\n\n\n$0\t\t\t\t\t\n2nd item\n\n\n\n\n\n$2.5\t...,danhash,$69.9,Escrow Yes,Dream,Danmark spain uk,...,2016,69.90,Drugs 42072,Drugs,[42072],"[7g, ORGANIC, Maroc, Hashish, sativa, ]","[, 7g, ORGANIC, Maroc, Hashish, sativa, ]",$0 2nd item $2.5 WorldMail (...,"[$0, $2.5]","[ , , , , , , , , , , , , n, d, , i, ..."
19,20.0,30 Pills Pain-O-Soma (Carisoprodol) 350mg | UK,Drugs \n\t\t40227,Pain-O-Soma contains Carisoprodol which is a m...,\n\n\n$9.87\t\t\t\t\t\nUK Tracked Post\n\n\n\n...,Mr.Erection,$28.36,Escrow Yes,Dream,UK,...,2016,28.36,Drugs 40227,Drugs,[40227],"[30, Pills, Pain, O, Soma, Carisoprodol, 350mg...","[, 30, Pills, Pain, O, Soma, Carisoprodol, 350...",$9.87 UK Tracked Post $12.3 ...,"[$9.87, $12.3]","[ , , , , , , , , , , , , , , U, K, ,..."
20,21.0,â 15GR â The BEST Quality Cocaine,Drugs \n\t\t40284,"Notice ""Constant Amazing quality is hard to ge...",\n\n\n$12.6\t\t\t\t\t\nDecoy Worldwide\n\n\n\n...,WarnerBos,$992,Escrow No,Dream,Europe,...,2016,992.00,Drugs 40284,Drugs,[40284],"[, 15GR, The, BEST, Quality, Cocaine]","[, 15GR, The, BEST, Quality, Cocaine, ]",$12.6 Decoy Worldwide $33.6 ...,"[$12.6, $33.6]","[ , , , , , , , , , , , , , , D, e, c,..."
21,22.0,40gr FLAKES COCAINE 92% â¯GAâ¯,Drugs \n\t\t41729,Gomorra Amsterdam Shop...,\n\n\n$4.74\t\t\t\t\t\nâ¯â¯Priorety Mailâ¯â...,gomorraamsterdam,$1687,Escrow No,Dream,EUROPE,...,2016,1687.00,Drugs 41729,Drugs,[41729],"[40gr, FLAKES, COCAINE, 92%, GA, ]","[, 40gr, FLAKES, COCAINE, 92%, GA, ]",$4.74 â¯â¯Priorety Mailâ¯â¯ â¯GAâ...,[$4.74],"[ , , , , , , , , , , , , , , â, , ¯,..."
22,23.0,250gr DUTCH SPEED PASTE AAA+ â¯GAâ¯,Drugs \n\t\t41729,Gomorra Amsterdam Shop...,\n\n\n$4.74\t\t\t\t\t\nPRIORETY MAIL â¯GAâ¯\...,gomorraamsterdam,$474,Escrow No,Dream,Netherlands,...,2016,474.00,Drugs 41729,Drugs,[41729],"[250gr, DUTCH, SPEED, PASTE, AAA, GA, ]","[, 250gr, DUTCH, SPEED, PASTE, AAA, GA, ]",$4.74 PRIORETY MAIL â¯GAâ¯,[$4.74],"[ , , , , , , , , , , , , , , P, R, I,..."
23,24.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,...,2016,NaN,-,NaN,[nan],"[Dream, Market, Login, Featured, anonymous, ma...","[, Dream, Market, Login, Featured, anonymous, ...",-,[],"[-, ]"
24,25.0,4 pill x Oxycontin 80mg ( PROMO DEAL ),Drugs \n\t\t41731,Mundi Pharma\nOxycontin 80mg,\n\n\n$5.22\t\t\t\t\t\nWorld\n\n\n\n\n\n$3.129...,doctorH1,$104.4,Escrow Yes,Dream,Germany,...,2016,104.40,Drugs 41731,Drugs,[41731],"[4, pill, x, Oxycontin, 80mg, PROMO, DEAL, ]","[, 4, pill, x, Oxycontin, 80mg, PROMO, DEAL, ]",$5.22 World $3.129 Europe,"[$5.22, $3.129]","[ , , , , , , , , , , , , , , W, o, r,..."


In [16]:

products_v2['desc_weight_grams'] = products_v2['description'].str.findall(r'[\d][g]')

In [17]:
products_v2.iloc[11:20, :]

,seqNo,product_name,category,description,shipping_options,seller_name,price,payment_method,market_name,ship_from,...,adj_price,category_revised,category_str,category_num,prod_name_split,prod_name_split_test,shipping_cleaned,shipping_costs,shipping_non_financial,desc_weight_grams
11,12.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,Dream,-,...,NaN,-,NaN,[nan],"[503, Forwarding, failure, Privoxy, localhost, ]","[, 503, Forwarding, failure, Privoxy, localhos...",-,[],"[-, ]",[]
12,13.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,...,NaN,-,NaN,[nan],"[Dream, Market, Login, Featured, anonymous, ma...","[, Dream, Market, Login, Featured, anonymous, ...",-,[],"[-, ]",[]
13,14.0,ORANGEBUD 5 GRAM,Drugs \n\t\t41829,"First bred in 1980s by Dutch Passion, Orange B...",\n\n\n$0\t\t\t\t\t\nPriority shipping Western ...,DutchDreams,$58.7,Escrow Yes,Dream,netherlands,...,58.70,Drugs 41829,Drugs,[41829],"[ORANGEBUD, 5, GRAM, ]","[, ORANGEBUD, 5, GRAM, ]",$0 Priority shipping Western and North...,"[$0, $0, $0, $3.147]","[ , , , , , , , , , , , P, r, i, o, r,...",[]
14,15.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,...,NaN,-,NaN,[nan],"[Dream, Market, Login, Featured, anonymous, ma...","[, Dream, Market, Login, Featured, anonymous, ...",-,[],"[-, ]",[]
15,16.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,...,NaN,-,NaN,[nan],"[Dream, Market, Login, Featured, anonymous, ma...","[, Dream, Market, Login, Featured, anonymous, ...",-,[],"[-, ]",[]
16,17.0,Fluor-Modafinil 100g,Drugs \n\t\t41830,Lab tested product purity around 98%.\nDo not ...,\n\n\n$39\t\t\t\t\t\nEMS\n\n\n\n\n\n$59\t\t\t\...,ROCKETCHEM,$490,Escrow Yes,Dream,China,...,490.00,Drugs 41830,Drugs,[41830],"[Fluor, Modafinil, 100g]","[, Fluor, Modafinil, 100g, ]",$39 EMS $59 Fedex/TNT,"[$39, $59]","[ , , , , , , , , , , , , E, M, S, , ...",[]
17,18.0,Listing not found,-,-,-,-,-,-,Dream,-,...,NaN,-,NaN,[nan],"[Listing, not, found]","[, Listing, not, found, ]",-,[],"[-, ]",[]
18,19.0,7g ORGANIC Maroc Hashish (sativa),Drugs \n\t\t42072,ONLY 9.99 per GRAM !!!\n\nOrganic Moroccan has...,\n\n\n$0\t\t\t\t\t\n2nd item\n\n\n\n\n\n$2.5\t...,danhash,$69.9,Escrow Yes,Dream,Danmark spain uk,...,69.90,Drugs 42072,Drugs,[42072],"[7g, ORGANIC, Maroc, Hashish, sativa, ]","[, 7g, ORGANIC, Maroc, Hashish, sativa, ]",$0 2nd item $2.5 WorldMail (...,"[$0, $2.5]","[ , , , , , , , , , , , , n, d, , i, ...",[]
19,20.0,30 Pills Pain-O-Soma (Carisoprodol) 350mg | UK,Drugs \n\t\t40227,Pain-O-Soma contains Carisoprodol which is a m...,\n\n\n$9.87\t\t\t\t\t\nUK Tracked Post\n\n\n\n...,Mr.Erection,$28.36,Escrow Yes,Dream,UK,...,28.36,Drugs 40227,Drugs,[40227],"[30, Pills, Pain, O, Soma, Carisoprodol, 350mg...","[, 30, Pills, Pain, O, Soma, Carisoprodol, 350...",$9.87 UK Tracked Post $12.3 ...,"[$9.87, $12.3]","[ , , , , , , , , , , , , , , U, K, ,...",[]


In [18]:
products_v2['category_str'].unique()

array([nan, 'Drugs ', 'Digital Goods ', 'Services ', 'Other ',
       'Drugs Paraphernalia ', 'Drugs Stimulants Meth',
       'Drugs Ecstasy MDMA', 'Drugs Cannabis Hash',
       'Drugs Stimulants Speed', 'Drugs Opioids Opium',
       'Digital Goods Erotica', 'Digital Goods EBooks',
       'Digital Goods Information eBooks', 'Drugs Stimulants',
       'Drugs Psychedelics LSD', 'Drugs Stimulants Cocaine',
       'Drugs Cannabis Seeds', 'Drugs Cannabis Concentrates',
       'Drugs Cannabis Edibles', 'Drugs Prescription', 'Drugs Opioids',
       'Drugs Cannabis Weed', 'Other Jewellery', 'Digital Goods',
       'Drugs Cannabis', 'Digital Goods Information Guides',
       'Digital Goods Hacking', 'Digital Goods Fraud',
       'Drugs Opioids Fentanyl', 'Drugs Benzos', 'Drugs Ecstasy',
       'Other Electronics', 'Drugs Opioids Oxycodone',
       'Services Cash out', 'Drugs Prescription Stimulants',
       'Digital Goods Drugs', 'Digital Goods Fraud Related CC CVV',
       'Digital Goods Fraud

In [19]:
products_v2['shipping_options_revised'] = products_v2['shipping_options'].str.replace(r'\n', ' ')
products_v2['shipping_options_revised'] = products_v2['shipping_options_revised'].str.replace('\t', ' ')

In [20]:
for i in products_v2.iloc[11:20, -1:].values:
    print(i)

['-']
['-']
['   $0      Priority shipping Western and Northern Europe      $0      Priority shipping  Eastern and  Southern Europe      $0       combined order only when ordering mulitiple strains to prevent paying twice      $3.147      Priority shipping  Eastern and  Southern Europe      ']
['-']
['-']
['   $39      EMS      $59      Fedex/TNT      ']
['-']
['   $0      2nd item      $2.5      WorldMail (World) 5DD      ']
['   $9.87      UK Tracked Post      $12.3      EU Tracked Post      ']


In [21]:
for x in products['description'][50: 70]:
    print(x)

5 gram pure SPEEDPASTE 72-74% PURITY - FREE SHIPMENT + TRACK&TRACE 
Lemon Haze is a hybrid of Lemon Skunk and Amnesia Haze, both of which are sativas that feature a prominent lemon flavor and aroma of their own. It has one of the higher THC contents at roughly 25 % with a CBD content of around 0.88 percent. Indeed, the strain is so potent that it won the 2008 High Times Cannabis Cup, which greatly increased its popularity.


Lemon Haze has been characterized as smelling of fresh peeled lemon slices with a similar taste. Its buds appear to be green and yellow with amber hairs on the trichomes, giving it the yellow tint. 

The Lemon Haze strain is popular for  its ability to make people feel happy, euphoric, and uplifted.


"There are not many strains better than this one Jamie Oliver couldn't make a better tasting product, that shit was magical"


Enjoy....


* by ordering we expect you to have read and agreed with the terms on our profile

 
-
                        Gomorra Amsterdam 

## EDA

In [22]:
utils.stats_summary(products['adj_price'])

KeyError: 'adj_price'

In [ ]:
plt.figure(figsize=(25,8))
less_than_100 = products[products['adj_price'] < 100]['adj_price']
sns.distplot(less_than_100)
plt.title('Price Distribution of Products Less Than 100', fontsize= 20);
print('# - Goods Less Than $100: ', len(less_than_100))
print('% - Total Good Less Than $100: {:.1%}'.format(len(less_than_100) / products.shape[0]))